In [1]:
import pyspark as ps
from pyspark.sql import SparkSession as ss
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,TimestampType
import pandas as pd

In [2]:
spark = ss.builder.master("local[*]").appName("mide_spark").getOrCreate()

22/03/01 19:59:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.version

'3.0.3'

In [4]:
path = '/home/mide/data-analytical-engineering/week_5/homework/'

In [5]:
schema = StructType([
    StructField('hvfhs_license_num',StringType(),True),
    StructField('dispatching_base_num',StringType(),True),
    StructField('pickup_datetime',TimestampType(),True),
    StructField('dropoff_datetime',TimestampType(),True),
    StructField('PULocationID',IntegerType(),True),
    StructField('DOLocationID',IntegerType(),True),
    StructField('SR_Flag',StringType(),True)]
    )

In [5]:
df = spark.read.option("header", "true").schema(schema).csv(path + 'raw/fhvhv_tripdata_2021-02.csv')

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [6]:
df.repartition(24).write.parquet(path + 'pq/fhvhv_tripdata_2021-02/')

In [5]:
data = spark.read.parquet(path + 'pq/fhvhv_tripdata_2021-02/')

In [6]:
data.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [7]:
data.registerTempTable('fhl_data')

In [8]:
df_feb_count = spark.sql("""

SELECT
   DATE(pickup_datetime) AS date,
   COUNT(1)
FROM
   fhl_data
GROUP BY
   1
HAVING
   date = '2021-02-15'

""")

In [9]:
df_feb_count.show()

+----------+--------+
|      date|count(1)|
+----------+--------+
|2021-02-15|  367170|
+----------+--------+



In [10]:
df_feb_duration = spark.sql("""

SELECT
   DATE(pickup_datetime) AS date,
   (bigint(to_timestamp(dropoff_datetime))) - (bigint(to_timestamp(pickup_datetime))) as time_diff
FROM
   fhl_data
ORDER BY
   time_diff DESC
LIMIT 1

""")

In [11]:
df_feb_duration.show()

+----------+---------+
|      date|time_diff|
+----------+---------+
|2021-02-11|    75540|
+----------+---------+



In [12]:
df_feb_dispatching_base_num = spark.sql("""

SELECT
   dispatching_base_num,
   COUNT(1) AS frequency
FROM
   fhl_data
GROUP BY
   1
ORDER BY
   frequency DESC
LIMIT 1
   


""")

In [13]:
df_feb_dispatching_base_num.show()

+--------------------+---------+
|dispatching_base_num|frequency|
+--------------------+---------+
|              B02510|  3233664|
+--------------------+---------+



In [15]:
zone = spark.read.parquet('/home/mide/data-analytical-engineering/week_5/data/zone/')

In [16]:
zone.registerTempTable('zone')

In [17]:
zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [42]:
pu_join = data.join(zone, data.PULocationID == zone.LocationID).drop('pickup_datetime', 'dropoff_datetime','SR_Flag','dispatching_base_num','Borough','service_zone','hvfhs_license_num')

In [43]:
pu_join = pu_join.withColumnRenamed('Zone', 'pu_zone')


In [44]:
pu_join.show()

+------------+------------+----------+--------------------+
|PULocationID|DOLocationID|LocationID|             pu_zone|
+------------+------------+----------+--------------------+
|         163|         235|       163|       Midtown North|
|         225|         181|       225|  Stuyvesant Heights|
|         260|          95|       260|            Woodside|
|         235|          60|       235|University Height...|
|          55|          55|        55|        Coney Island|
|          74|          81|        74|   East Harlem North|
|         239|         231|       239|Upper West Side S...|
|          95|         196|        95|        Forest Hills|
|         241|         265|       241|Van Cortlandt Vil...|
|          90|         137|        90|            Flatiron|
|           7|           7|         7|             Astoria|
|         231|         246|       231|TriBeCa/Civic Center|
|         108|          29|       108|           Gravesend|
|          11|         112|        11|  

In [45]:
zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [46]:
du_join = pu_join.join(zone,pu_join.DOLocationID == zone.LocationID).drop('service_zone','hvfhs_license_num')

In [47]:
du_join.show()

+------------+------------+----------+------------+----------+-------------+--------------------+
|PULocationID|DOLocationID|LocationID|     pu_zone|LocationID|      Borough|                Zone|
+------------+------------+----------+------------+----------+-------------+--------------------+
|          55|          55|        55|Coney Island|         1|          EWR|      Newark Airport|
|          55|          55|        55|Coney Island|         2|       Queens|         Jamaica Bay|
|          55|          55|        55|Coney Island|         3|        Bronx|Allerton/Pelham G...|
|          55|          55|        55|Coney Island|         4|    Manhattan|       Alphabet City|
|          55|          55|        55|Coney Island|         5|Staten Island|       Arden Heights|
|          55|          55|        55|Coney Island|         6|Staten Island|Arrochar/Fort Wad...|
|          55|          55|        55|Coney Island|         7|       Queens|             Astoria|
|          55|      

In [49]:
play = du_join.select('pu_zone','Zone')

In [50]:
play.show()

+------------+--------------------+
|     pu_zone|                Zone|
+------------+--------------------+
|Coney Island|      Newark Airport|
|Coney Island|         Jamaica Bay|
|Coney Island|Allerton/Pelham G...|
|Coney Island|       Alphabet City|
|Coney Island|       Arden Heights|
|Coney Island|Arrochar/Fort Wad...|
|Coney Island|             Astoria|
|Coney Island|        Astoria Park|
|Coney Island|          Auburndale|
|Coney Island|        Baisley Park|
|Coney Island|          Bath Beach|
|Coney Island|        Battery Park|
|Coney Island|   Battery Park City|
|Coney Island|           Bay Ridge|
|Coney Island|Bay Terrace/Fort ...|
|Coney Island|             Bayside|
|Coney Island|             Bedford|
|Coney Island|        Bedford Park|
|Coney Island|           Bellerose|
|Coney Island|             Belmont|
+------------+--------------------+
only showing top 20 rows



In [51]:
play.registerTempTable('play')

In [65]:
df_df_pu = spark.sql("""

SELECT 
    CONCAT(CAST(pu_zone AS VARCHAR(50)), '/', CAST(Zone AS VARCHAR(50))) AS pickup_dropoff,
    COUNT(*) AS rides
FROM
    play
GROUP BY
    pickup_dropoff
ORDER BY
    rides DESC

    

""")

In [66]:
df_df_pu.show()

+--------------------+------+
|      pickup_dropoff| rides|
+--------------------+------+
|East New York/Gov...|135123|
|Borough Park/Gove...|111987|
|East New York/Corona| 90082|
|Canarsie/Governor...| 84078|
|Crown Heights Nor...| 77928|
| Borough Park/Corona| 74658|
|     Canarsie/Corona| 56052|
|Bay Ridge/Governo...| 53802|
|Crown Heights Nor...| 51952|
|East New York/Uni...| 45041|
|East New York/Cen...| 45041|
|East New York/Lon...| 45041|
|East New York/Spr...| 45041|
|East New York/Red...| 45041|
|East New York/Rid...| 45041|
|East New York/Mid...| 45041|
|East New York/Wil...| 45041|
|East New York/Wes...| 45041|
|East New York/Gre...| 45041|
|East New York/Cro...| 45041|
+--------------------+------+
only showing top 20 rows

